### glacier elevation change tiles processing
1. mosaic of images;   
2. clip the mosaic image to hma region.


In [ ]:
import os
import rasterio as rio
import geopandas as gpd
from glob import glob
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from rasterio.warp import calculate_default_transform


### Mosaic

In [18]:
paths_tile_ls = glob('data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/*dhdt.tif')
path_mosaic = 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/dhdt_mosaic.tif'
len(paths_tile_ls)


320

In [16]:
def raster_reproj(src, target_crs="EPSG:4326", resample_method=Resampling.nearest):
    """
    Reproject a raster to a specified CRS.
    Parameters:
        src (rasterio.io.DatasetReader): Source raster.
        target_crs (str): Target CRS in EPSG format.
        resample_method (Resampling): Resampling method to use.        
    Returns:
        src_reproj (numpy.ndarray): Reprojected raster data.
    """
    transform, width, height = calculate_default_transform(
                src.crs, target_crs, src.width, src.height, *src.bounds)    
    kwargs = src.profile.copy()
    kwargs.update({
        'crs': target_crs, 'transform': transform,
        'width': width, 'height': height})
    
    memfile = rio.io.MemoryFile()
    with memfile.open(**kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=resample_method
            )
        src_reproj = memfile.open()  # Open the memory file as a rasterio dataset
    return src_reproj

src = rio.open(paths_tile_ls[0])
dst_crs = "EPSG:4326"
src_reproj = raster_reproj(src, dst_crs)
src_reproj.read()


array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]],
      shape=(1, 97, 221), dtype=float32)

In [17]:
srcs_to_mosaic = []
for fp in paths_tile_ls:
    src = rio.open(fp)
    src_reproj2wgs84 = raster_reproj(src, target_crs="EPSG:4326")
    srcs_to_mosaic.append(src_reproj2wgs84)

mosaic_arr, mosaic_trans = merge(srcs_to_mosaic)
mosaic_meta = src.meta.copy()
mosaic_meta.update({
    "height": mosaic_arr.shape[1],
    "width": mosaic_arr.shape[2],
    "transform": mosaic_trans,
    "crs": "EPSG:4326"  # Ensure the CRS is set to WGS84
    })
# # Write the mosaic raster to disk
# with rio.open(path_mosaic, 'w', **mosaic_meta) as dest:
#     dest.write(mosaic_arr)


### clip image to subregion

In [ ]:
path_dhdt = 'data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/dhdt_mosaic.tif'
path_hma_gtng = 'data/hma-extent/HMA/hma_gtng_202307_subregions.gpkg'
hma_vec_gdf = gpd.read_file(path_hma_gtng)

with rio.open(path_dhdt) as src:
    if hma_vec_gdf.crs != src.crs: 
        hma_vec_gdf = hma_vec_gdf.to_crs(src.crs)      
    for idx, row in hma_vec_gdf.iterrows():
        geom = row.geometry
        clipped_arr, clipped_transform = mask(dataset=src, shapes = [geom], 
                                              crop=True, all_touched=True)
        meta = src.meta.copy()
        meta.update({
            "height": clipped_arr.shape[1],
            "width": clipped_arr.shape[2],
            "transform": clipped_transform})
        # ## save to path
        name_subregion = row['full_name'].split(' (')[0].replace(' ', '_').lower()
        path_save = f"data/glacier-height/hugonnet-gla-dset/dhdt_2000_2019/dhdt/hma-subregions/dhdt_{name_subregion}.tif"
        # if os.path.exists(path_save): os.remove(path_save)
        # with rio.open(path_save, "w", **meta) as dst:
        #     dst.write(clipped_arr)
        # print(f"saved to: {path_save}")
